<a target="_blank" href="https://colab.research.google.com/github/jmanuelc87/nmp-autoavanza/blob/main/notebooks/MontePiedad_Extraction.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import base64
from PIL import Image

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate

from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
path = "OpenGVLab/InternVL2_5-1B"

# Extract Information

In [ ]:
model = AutoModelForCausalLM.from_pretrained(path, load_in_8bit=True, trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

In [ ]:
pipeline_llm = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)

In [ ]:
messages = [
    {
        "role": "user",
        "content": [{"type": "text", "text": "Extrae el texto de la imagen."}, {"type": "image_url", "image_url": "https://drive.usercontent.google.com/download?id=1J1dUqhxgq_uB8rNvIkqJ04HlZPLL5lbW"}]
    },
]

In [ ]:
output = pipeline_llm(text_inputs=messages, max_new_tokens=200)

In [ ]:
import pprint

pprint.pprint(output[0]["generated_text"][-1]["content"])